<a href="https://www.kaggle.com/code/aisuko/encoder-in-transformer-architecture?scriptVersionId=163273691" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Let's talks about the **Encoder** part in Transformers

**All the pictures are from the blogs in the Credit section bottom or internet, and the results may have some mistakes. It is also good to illustrate the details of calculation processes.**


<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/910/929/370/732/small/00d39d49cc82031d.png" width="100%" heigh="100%" alt="encoder in transformers "></div>

In the encoder, we perform complex operations involving matrices of queries, keys, and values. These operations are crucial for transforming the input data and extracting meaningful representations.


# What is attention?

The concept of **attention** in deep learning has its roots in the effort to improve RNNs for handling longer sequences or sentences. For instance, consider translating a sentence from one language to another. Translating a sentence word-by-word is usually not an opion because it ignores the complex grammatical structures and idiomatic expressions unique to each language, leading to inaccurate or nonsensical translations.

<div style="text-align: center"><img src="https://files.mastodon.social/media_attachments/files/111/819/375/137/031/595/small/4d1bab47ca3d3554.webp" width="100%" heigh="100%" alt="attention"></div>

To overcome this issue, attention mechanisms were introduced to give access to all sequence elements at each time step. The key is to be selective and determine which words are most important in a specific context.

**Which enables the network to understand which parts of the input sequence are the most relevant for the given task?**

For each token in the sequence, the attention mechanism identifies which other tokens are in the important for understanding the current token in the given context. See [Neural Network Architecture(Transformers)](https://www.kaggle.com/code/aisuko/neural-network-architecture-transformers) to get how we do the encode and embeddings to the inputs.


## Intuition of the concept

One way to understand attention is to think of it as a method which replaces each token embedding with an embedding that includes information about its neighbouring tokens; instead of using the same embedding for every token regardless of its context, it is knowned as `contextulised embeddings`, this is ultimately what we are trying to create.

If we knew which tokens were relevant to the current token, one way of capturing this context would be to create a **weighted average** - or, more generally, a linear combination - of these embeddings.

# Multi Head Attention in Transformers

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/603/805/414/005/410/small/a5f56f1e79523ecb.png" width="80%" heigh="50%" alt="multiplication_attention_scores"></div>


Here is a multi attention, let's use a single head calculation process as an example show the whole how we deal with Tensors(Embedding vector). As you can see above, there are three inputs in attention layer:
* Query
* Key
* Value

Here is an beautiful picture from [fareedkhandev's article](https://medium.com/@fareedkhandev/understanding-transformers-a-step-by-step-math-example-part-1-a7809015150a) can help us to understand the calculation process.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/603/890/432/233/262/small/5f997e34fc8c93db.webp" width="80%" heigh="80%" alt="calculation example"></div>

As we can see, the three input matrices(pink matrices) represent the transposed output obtained from the previously steps of adding position embeddings to the worlds embedding matrix. And the linear weights matrices(yellow, blue and red) represent the weight used in the attention mechanism. These matrices can have any number of dimention of the columns, but the number of row must be same as the number of columns in the input matrices for matrix multiplication.

Let's assume that the linear weights(yellow, blue and red) are adjusted during the training process through techniques like backpropogration and gradient descent. We can do the matrix multiplications below:

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/603/992/766/474/377/small/5df72b068852f4da.webp" width="80%" heigh="80%" alt="inputs combined embeddings and linear weights multiplications"></div>

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/604/011/854/311/850/small/cff738ddc12db575.webp" width="80%" heigh="80%" alt="calculation of mat mut"></div>


### Scale before Softmax

To ensure that our weights sum to one, we also apply the [SoftMax function](https://en.wikipedia.org/wiki/Softmax_function). However, as matrix multiplacations can produce arbitrarily large numbers, this could result in the SoftMax function returning very small gradients for large attention scores; which may lead to the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem) during training. To counteract this, the attention sores are multiplied by a scaling factor, before applying the SoftMax.



<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/604/029/599/828/338/small/0ed70349862bfb10.webp" width="100%" heigh="100%" alt="calculation of scale"></div>

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/604/040/441/989/075/small/98702165af7df12c.webp" width="100%" heigh="100%" alt="calculation of Softmax"></div>




### Concatenation Single/Multi-Head Attention

According to the paper, the authors found that it beneficial to run attention function with different learning rate in parallel with `h` times. This is the reason we need MHA(Multi-head), it allows the model to joinly attened to informaion from dufferent representation subspaces at different positions. "And due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality".

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/604/079/683/617/150/small/d329b118c10168d4.webp" width="100%" heigh="100%" alt="concatenation single/mul-head-attention calculation calculation results"></div>

### Linear transformation

We will once again perform a linear transformation similar to the process used to obtain query, key and value matrices. And this linear transformation is applied to the concatenated matrix obtained from xxx-head-attentions.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/604/075/669/870/340/small/8153273908cbaa73.webp" width="100%" heigh="100%" alt="linear transformation"></div>

# Adding and Normalizing

In **Add & Norm*** section, we need to add the result of `multi-head-attention` to `positional encoder embedding vector`. For example,

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/524/819/349/331/small/1102d4173a7dbe8f.webp" width="100%" heigh="100%" alt="adding metrics to perform add"></div>


## Normalizing the matrix

To normalize the above matrix, we need to compute the mean and strandard deviation row-wise for each row

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/562/980/146/965/small/d7fde74bfc346a3e.webp" width="100%" heigh="100%" alt="calculating means and std"></div>


Here, we subtract each value of the matrix by the corresponding row mean and divide it by the corresponding standard deviation. Add a small value of errors prevents the denominator from beng zero nd avoid making the entire term infinity.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/643/247/661/001/small/a3b75283e5c3c8ab.webp" width="100%" heigh="100%" alt="normalizing the resultant matrix"></div>

# Feed Forward Network

After normalizing the matrix, it will be processed through a feedforward network. We will be using a very basic network that contains only one linear layer and one ReLU activation function layer like below:

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/706/307/822/936/small/ce8603262a93044e.webp" width="100%" heigh="100%" alt="normalizing the resultant matrix"></div>


## Calculate the Linear layer

We need to calculate the linear layer by multiplying our last calculated matrix with a random set of weights matrix, which will be updated when the transformer starts learning, and adding the resultant matrix to a bias matrix that also contains random values.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/759/068/477/669/small/d1414532c3975e1d.webp" width="100%" heigh="100%" alt="calculating Linear Layer"></div>


## Calculate the ReLU layer

After calculating the linear layer, we need to pass it through the ReLU layer and use its formula.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/781/343/803/316/original/8bfd8ecbc1f2e0c5.webp" width="100%" heigh="100%" alt="calculating Linear Layer"></div>



# Adding and Normalizing Again

Once we obtain the resultant matrix from feed for forward network, we have to add it to the matrix that is obtained from previous add and norm step, and then normalizing it using the **row wise mean** and **standard deviation**.

<div style="text-align: center"><img src="https://hostux.social/system/media_attachments/files/111/609/823/455/908/192/small/2ea27512ea50ad89.webp" width="100%" heigh="100%" alt="add and norm after feed forward network"></div>


The output matrix of this step will serve as the query and key matrix in one of the multi-head attention mechanisms present in [Decoder in Transformer](https://www.kaggle.com/code/aisuko/decoder-in-transformer-architecture/notebook).

# Credit
* https://medium.com/towards-data-science/de-coded-transformers-explained-in-plain-english-877814ba6429
* https://medium.com/@fareedkhandev/understanding-transformers-a-step-by-step-math-example-part-1-a7809015150a
* https://medium.com/@fareedkhandev/understanding-transformers-from-start-to-end-a-step-by-step-math-example-16d4e64e6eb1
* https://huggingface.co/blog/getting-started-with-embeddings
* https://huggingface.co/bert-base-uncased
* https://magazine.sebastianraschka.com?utm_source=navbar&utm_medium=web&r=fbe14